### Imports

In [1]:
# COMMON LIBRARIES
import os
import cv2
import json
import torch

from datetime import datetime

# DATA SET PREPARATION AND LOADING
from detectron2.data.datasets import register_coco_instances
from detectron2.data import DatasetCatalog, MetadataCatalog

# VISUALIZATION
from detectron2.utils.visualizer import Visualizer
from detectron2.utils.visualizer import ColorMode

# CONFIGURATION
from detectron2 import model_zoo
from detectron2.config import get_cfg

# EVALUATION
from detectron2.engine import DefaultPredictor

# TRAINING
from detectron2.engine import DefaultTrainer

### For Dataset in roboflow

In [2]:
import os
from roboflow import Roboflow
from azure.storage.blob import BlobServiceClient
from azure.core.exceptions import ResourceExistsError  # Correct import


rf = Roboflow(api_key="p7ll5rTRQYfRRG5zTZYK")
project = rf.workspace("amodic2c").project("paddy_weed")
version = project.version(1)
dataset = version.download("coco-segmentation")
                
dataset_path = dataset.location  # Correct way to get the path

# # Azure Blob Storage credentials
# 
# # Initialize Blob Service Client
# blob_service_client = BlobServiceClient.from_connection_string(AZURE_CONNECTION_STRING)
# container_client = blob_service_client.get_container_client(CONTAINER_NAME)

# # Ensure the container exists
# try:
#     container_client.create_container()
#     print(f"Created container: {CONTAINER_NAME}")
# except ResourceExistsError:
#     print(f"Container '{CONTAINER_NAME}' already exists.")

# # Upload dataset files to Azure Blob Storage
# for root, dirs, files in os.walk(dataset_path):  # Use correct dataset path
#     for file_name in files:
#         file_path = os.path.join(root, file_name)
#         blob_name = os.path.relpath(file_path, dataset_path)  # Preserve folder structure in blob

#         # Upload file
#         blob_client = container_client.get_blob_client(blob_name)
#         with open(file_path, "rb") as data:
#             blob_client.upload_blob(data, overwrite=True)
#         print(f"Uploaded: {blob_name}")

# print("Dataset successfully uploaded to Azure Blob Storage!")


loading Roboflow workspace...
loading Roboflow project...


In [3]:
import json

annotation_file = "/home/prajjwal/Paddy_weed_detection/paddy_weed-1/test/_annotations_fixed.coco.json"

with open(annotation_file, "r") as f:
    data = json.load(f)

category_ids = {c["id"]: c["name"] for c in data["categories"]}
print("Category Mapping:", category_ids)


Category Mapping: {0: 'weed', 1: 'Paddy'}


In [4]:
import json

annotation_file = "/home/prajjwal/Paddy_weed_detection/paddy_weed-1/train/_annotations_fixed.coco.json"  # Adjust for your dataset format
with open(annotation_file, "r") as f:
    data = json.load(f)

class_counts = set()
for annotation in data["annotations"]:
    class_counts.add(annotation["category_id"])

print(f"Unique class IDs in dataset: {class_counts}")

Unique class IDs in dataset: {0, 1}


In [5]:
import json

# Path to your COCO annotation file
json_path = "/home/prajjwal/Paddy_weed_detection/paddy_weed-1/valid/_annotations.coco.json"

# Load the existing COCO JSON
with open(json_path, "r") as f:
    data = json.load(f)

# Step 1: Fix category mapping
# Keep only "weed" (ID=0) and "Paddy" (ID=1), remove "Weed" (ID=2)
new_categories = [
    {"id": 0, "name": "weed", "supercategory": "none"},
    {"id": 1, "name": "Paddy", "supercategory": "none"}
]

# Step 2: Update annotations - Change all instances of "Weed" (ID=2) to "weed" (ID=0)
for ann in data["annotations"]:
    if ann["category_id"] == 2:
        ann["category_id"] = 0  # Reassign to "weed" ID=0

# Step 3: Replace the categories section
data["categories"] = new_categories

# Save the fixed JSON
fixed_json_path = "/home/prajjwal/Paddy_weed_detection/paddy_weed-1/valid/_annotations_fixed.coco.json"
with open(fixed_json_path, "w") as f:
    json.dump(data, f, indent=4)

print(f"Fixed COCO annotations saved to {fixed_json_path}")


Fixed COCO annotations saved to /home/prajjwal/Paddy_weed_detection/paddy_weed-1/valid/_annotations_fixed.coco.json


In [6]:
DATA_SET_NAME = dataset_path.split("/")[-1].replace(" ", "-")
ANNOTATIONS_FILE_NAME = "_annotations_fixed.coco.json"

In [7]:
# TRAIN SET
TRAIN_DATA_SET_NAME = f"{DATA_SET_NAME}-train"
TRAIN_DATA_SET_IMAGES_DIR_PATH = os.path.join(dataset_path, "train")
TRAIN_DATA_SET_ANN_FILE_PATH = os.path.join(dataset_path, "train", ANNOTATIONS_FILE_NAME)

register_coco_instances(
    name=TRAIN_DATA_SET_NAME, 
    metadata={}, 
    json_file=TRAIN_DATA_SET_ANN_FILE_PATH, 
    image_root=TRAIN_DATA_SET_IMAGES_DIR_PATH
)

# TEST SET
TEST_DATA_SET_NAME = f"{DATA_SET_NAME}-test"
TEST_DATA_SET_IMAGES_DIR_PATH = os.path.join(dataset_path, "test")
TEST_DATA_SET_ANN_FILE_PATH = os.path.join(dataset_path, "test", ANNOTATIONS_FILE_NAME)

register_coco_instances(
    name=TEST_DATA_SET_NAME, 
    metadata={}, 
    json_file=TEST_DATA_SET_ANN_FILE_PATH, 
    image_root=TEST_DATA_SET_IMAGES_DIR_PATH
)

# VALID SET
VALID_DATA_SET_NAME = f"{DATA_SET_NAME}-valid"
VALID_DATA_SET_IMAGES_DIR_PATH = os.path.join(dataset_path, "valid")
VALID_DATA_SET_ANN_FILE_PATH = os.path.join(dataset_path, "valid", ANNOTATIONS_FILE_NAME)

register_coco_instances(
    name=VALID_DATA_SET_NAME, 
    metadata={}, 
    json_file=VALID_DATA_SET_ANN_FILE_PATH, 
    image_root=VALID_DATA_SET_IMAGES_DIR_PATH
)

In [8]:
[
    data_set
    for data_set
    in MetadataCatalog.list()
    if data_set.startswith(DATA_SET_NAME)
]

['paddy_weed-1-train', 'paddy_weed-1-test', 'paddy_weed-1-valid']

**TRAINING**

In [9]:
from detectron2.engine import HookBase, DefaultTrainer
from detectron2.config import get_cfg
from detectron2.solver import build_lr_scheduler
from detectron2 import model_zoo  # Corrected import
import torch
from typing import List, Dict, Optional

class LRSchedulerWithEarlyStopping(HookBase):
    def __init__(self, patience=5):
        self.patience = patience
        self.best_loss = float('inf')
        self.waiting = 0
        
    def after_step(self):
        stats = self.trainer.storage.latest()
        
        # Ensure we're accessing the total_loss or the correct loss value
        current_loss = stats['total_loss'] if isinstance(stats['total_loss'], (int, float)) else stats['total_loss'][0]
        
        if current_loss < self.best_loss:
            self.best_loss = current_loss
            self.waiting = 0
        else:
            self.waiting += 1
            
        if self.waiting >= self.patience:
            # Ensure the scheduler is initialized
            if self.trainer.scheduler is not None:
                self.trainer.scheduler.step()
            self.waiting = 0

class WeedTrainer(DefaultTrainer):
    def __init__(self, cfg):
        super().__init__(cfg)
        self.scheduler = build_lr_scheduler(cfg, self.optimizer)  # Initialize the scheduler
    
    def build_hooks(self):
        hooks = super().build_hooks()
        hooks.append(LRSchedulerWithEarlyStopping(patience=5))
        return hooks

def get_optimized_weed_detection_config():
    cfg = get_cfg()
    ARCHITECTURE = "mask_rcnn_X_101_32x8d_FPN_3x"  # Using ResNeXt101
    CONFIG_FILE_PATH = f"COCO-InstanceSegmentation/{ARCHITECTURE}.yaml"
    cfg.merge_from_file(model_zoo.get_config_file(CONFIG_FILE_PATH))
    cfg.MODEL.WEIGHTS = model_zoo.get_checkpoint_url(CONFIG_FILE_PATH)
    cfg.OUTPUT_DIR = "/home/prajjwal/Paddy_weed_detection/output"

    # Dataset Configuration
    cfg.DATASETS.TRAIN = ("paddy_weed-1-train",)
    cfg.DATASETS.TEST = ("paddy_weed-1-test",)
    cfg.DATASETS.VAL = ("paddy_weed-1-valid",)

    # Model Architecture
    cfg.MODEL.ROI_HEADS.NUM_CLASSES = 2
    cfg.MODEL.MASK_ON = True
    cfg.MODEL.DEVICE = "cuda" if torch.cuda.is_available() else "cpu"

    # Optimized Training Hyperparameters with LR Scheduler support
    cfg.SOLVER.IMS_PER_BATCH = 4
    cfg.SOLVER.BASE_LR = 0.0003
    cfg.SOLVER.MAX_ITER = 30000
    cfg.SOLVER.STEPS = (20000, 25000, 28000)
    cfg.SOLVER.GAMMA = 0.1
    cfg.SOLVER.WARMUP_FACTOR = 1.0 / 2000
    cfg.SOLVER.WARMUP_ITERS = 2000
    cfg.SOLVER.WARMUP_METHOD = "linear"
    cfg.SOLVER.WEIGHT_DECAY = 0.00025
    cfg.SOLVER.MOMENTUM = 0.9
    cfg.SOLVER.CLIP_GRADIENTS.ENABLED = True
    cfg.SOLVER.CLIP_GRADIENTS.CLIP_VALUE = 1.0
    cfg.SOLVER.CHECKPOINT_PERIOD = 1000 

    # ROI Head Configuration
    cfg.MODEL.ROI_HEADS.BATCH_SIZE_PER_IMAGE = 256
    cfg.MODEL.ROI_HEADS.POSITIVE_FRACTION = 0.25
    cfg.MODEL.ROI_HEADS.NMS_THRESH_TEST = 0.3
    cfg.MODEL.ROI_HEADS.SCORE_THRESH_TEST = 0.6
    cfg.MODEL.ROI_HEADS.IOU_THRESHOLDS = [0.3, 0.5, 0.7]
    cfg.MODEL.ROI_HEADS.IOU_LABELS = [0, 1, 1, 1]
    assert len(cfg.MODEL.ROI_HEADS.IOU_THRESHOLDS) + 1 == len(cfg.MODEL.ROI_HEADS.IOU_LABELS), "IoU thresholds and labels are mismatched."

    # RPN Configuration
    cfg.MODEL.RPN.BATCH_SIZE_PER_IMAGE = 256
    cfg.MODEL.RPN.POSITIVE_FRACTION = 0.5
    cfg.MODEL.RPN.PRE_NMS_TOPK_TRAIN = 6000
    cfg.MODEL.RPN.POST_NMS_TOPK_TRAIN = 1000
    cfg.MODEL.RPN.PRE_NMS_TOPK_TEST = 3000
    cfg.MODEL.RPN.POST_NMS_TOPK_TEST = 500
    cfg.MODEL.RPN.NMS_THRESH = 0.7

    # Anchor Generator
    cfg.MODEL.ANCHOR_GENERATOR.SIZES = [[16, 32, 64, 128, 256]]
    cfg.MODEL.ANCHOR_GENERATOR.ASPECT_RATIOS = [[0.25, 0.5, 1.0, 2.0]]

    # Input Configuration
    cfg.INPUT.MIN_SIZE_TRAIN = (640, 672, 704, 736, 768, 800)
    cfg.INPUT.MAX_SIZE_TRAIN = 1333
    cfg.INPUT.MIN_SIZE_TEST = 800
    cfg.INPUT.MAX_SIZE_TEST = 1333
    cfg.INPUT.MASK_FORMAT = "bitmask"
    
    # Data Augmentation
    cfg.INPUT.RANDOM_FLIP = "horizontal"
    cfg.INPUT.BRIGHTNESS = 1.0
    cfg.INPUT.CONTRAST = 1.0
    cfg.INPUT.SATURATION = 1.0
    cfg.INPUT.ROTATION = [-15, 15]
    
    # Dataloader
    cfg.DATALOADER.NUM_WORKERS = 4
    cfg.DATALOADER.FILTER_EMPTY_ANNOTATIONS = True
    cfg.DATALOADER.ASPECT_RATIO_GROUPING = True
    
    # Evaluation
    cfg.TEST.EVAL_PERIOD = 1000
    cfg.TEST.DETECTIONS_PER_IMAGE = 100
    cfg.TEST.PRECISE_BN.ENABLED = True
    cfg.TEST.PRECISE_BN.NUM_ITER = 200
    
    # Test-Time Augmentation
    cfg.TEST.AUG.ENABLED = True
    cfg.TEST.AUG.MIN_SIZES = (600, 700, 800, 900, 1000)
    cfg.TEST.AUG.MAX_SIZE = 1400
    cfg.TEST.AUG.FLIP = True
    
    # Mixed Precision Training
    cfg.SOLVER.AMP.ENABLED = False
    
    return cfg

# Usage example:

# cfg = get_optimized_weed_detection_config()
# trainer = WeedTrainer(cfg)
# trainer.resume_or_load(resume=True)
# trainer.train()

In [ ]:
import torch
import random
import os

from detectron2.engine import DefaultTrainer
from detectron2.config import get_cfg
from detectron2.data import DatasetCatalog, MetadataCatalog
from detectron2 import model_zoo

def setup_cfg():
    cfg = get_cfg()
    cfg.merge_from_file("/home/prajjwal/Paddy_weed_detection/config.yaml")
    cfg.MODEL.DEVICE = "cuda" if torch.cuda.is_available() else "cpu"
    cfg.OUTPUT_DIR = "/home/prajjwal/Paddy_weed_detection/output2"
    os.makedirs(cfg.OUTPUT_DIR, exist_ok=True)
    cfg.MODEL.WEIGHTS = "/home/prajjwal/Paddy_weed_detection/output/model_0013999.pth"
    return cfg

def balance_dataset():
    train_dataset = DatasetCatalog.get("paddy_weed-1-train")
    filtered_dataset = [data for data in train_dataset if "annotations" in data and data["annotations"]]
    weed_samples = [data for data in filtered_dataset if data["annotations"][0]["category_id"] == 0]
    paddy_samples = [data for data in filtered_dataset if data["annotations"][0]["category_id"] == 1]
    if not weed_samples or not paddy_samples:
        raise ValueError("Error: One of the classes (weed or paddy) has no samples after filtering. Check dataset.")
    random.shuffle(weed_samples)
    weed_samples = weed_samples[:len(paddy_samples)]
    balanced_dataset = weed_samples + paddy_samples
    random.shuffle(balanced_dataset)
    if "paddy_weed-1-train-balanced" in DatasetCatalog.list():
        DatasetCatalog.remove("paddy_weed-1-train-balanced")
    def get_balanced_dataset():
        return balanced_dataset
    DatasetCatalog.register("paddy_weed-1-train-balanced", get_balanced_dataset)
    MetadataCatalog.get("paddy_weed-1-train-balanced").set(thing_classes=["weed", "paddy"])
    print(f"Balanced Dataset: {len(weed_samples)} weed, {len(paddy_samples)} paddy")
    return "paddy_weed-1-train-balanced"

def restart_training():
    cfg = setup_cfg()
    balanced_dataset_name = balance_dataset()
    cfg.DATASETS.TRAIN = (balanced_dataset_name,)
    cfg.DATASETS.TEST = ("paddy_weed-1-test",)
    cfg.SOLVER.BASE_LR = 0.0003
    cfg.SOLVER.WARMUP_ITERS = 1000
    cfg.SOLVER.MAX_ITER = 16000
    cfg.MODEL.ROI_HEADS.NUM_CLASSES = 2
    cfg.MODEL.ROI_HEADS.SCORE_THRESH_TEST = 0.4
    cfg.MODEL.ROI_HEADS.NMS_THRESH_TEST = 0.3
    trainer = DefaultTrainer(cfg)
    trainer.resume_or_load(resume=True)
    print("Training restarted from Step 0 using the provided model checkpoint...")
    trainer.train()

if __name__ == "__main__":
    restart_training()



Category ids in annotations are not in [1, #categories]! We'll apply a mapping for you.



Balanced Dataset: 7473 weed, 7473 paddy
[02/10 10:03:37 d2.engine.defaults]: Model:
GeneralizedRCNN(
  (backbone): FPN(
    (fpn_lateral2): Conv2d(256, 256, kernel_size=(1, 1), stride=(1, 1))
    (fpn_output2): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (fpn_lateral3): Conv2d(512, 256, kernel_size=(1, 1), stride=(1, 1))
    (fpn_output3): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (fpn_lateral4): Conv2d(1024, 256, kernel_size=(1, 1), stride=(1, 1))
    (fpn_output4): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (fpn_lateral5): Conv2d(2048, 256, kernel_size=(1, 1), stride=(1, 1))
    (fpn_output5): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (top_block): LastLevelMaxPool()
    (bottom_up): ResNet(
      (stem): BasicStem(
        (conv1): Conv2d(
          3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False
          (norm): FrozenBatchNorm2d(num_features=64,

torch.meshgrid: in an upcoming release, it will be required to pass the indexing argument. (Triggered internally at /pytorch/aten/src/ATen/native/TensorShape.cpp:3637.)


[02/10 10:09:41 d2.utils.events]:  eta: 3 days, 8:20:43  iter: 19  total_loss: 0.9566  loss_cls: 0.07711  loss_box_reg: 0.35  loss_mask: 0.2105  loss_rpn_cls: 0.04868  loss_rpn_loc: 0.2121    time: 17.9278  last_time: 17.6882  data_time: 0.0170  last_data_time: 0.0088   lr: 5.8471e-06  


### Tensorflow training curves

In [ ]:
# Look at training curves in tensorboard:
%load_ext tensorboard
%tensorboard --logdir $/content/drive/MyDrive/ColabOutputs

In [ ]:
cfg.MODEL.WEIGHTS = os.path.join(cfg.OUTPUT_DIR, "model_final.pth")
cfg.MODEL.ROI_HEADS.SCORE_THRESH_TEST = 0.7
predictor = DefaultPredictor(cfg)

### Save config

In [21]:
f = open('config.yaml', 'w')
f.write(cfg.dump())
f.close()